# Nature DQN to Play Atari

Unofficial implementation of
- V. Minh, et. al, “Human-level control through deep reinforcement learning”, Nature 2015.

In [1]:
%matplotlib inline
import os
import sys
import time
import itertools
import logging

import numpy as np
np.random.seed(0)
import pandas as pd
import gym
import tensorflow as tf
from tensorflow import keras
from PIL import Image
import matplotlib.pyplot as plt

logging.basicConfig(stream=sys.stdout, level=logging.DEBUG,
        format='%(asctime)s [%(levelname)s] %(message)s')

2019-02-13 08:50:51,770 [DEBUG] Loaded backend module://ipykernel.pylab.backend_inline version unknown.


In [2]:
# env_spec_id = 'BreakoutDeterministic-v4'
# env_spec_id = 'PongDeterministic-v4'
# env_spec_id = 'SeaquestDeterministic-v4'
env_spec_id = 'SpaceInvadersDeterministic-v4'
# env_spec_id = 'BeamRiderDeterministic-v4'
env = gym.make(env_spec_id)
print('observation_space = {}'.format(env.observation_space))
print('action_space = {}'.format(env.action_space))
print(env.__dict__)
env.seed(0)

D:\Programs\Anaconda3\envs\base36\lib\site-packages\gym\envs\registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


observation_space = Box(210, 160, 3)
action_space = Discrete(6)
{'env': <gym.envs.atari.atari_env.AtariEnv object at 0x0000027E9B79A4E0>, 'action_space': Discrete(6), 'observation_space': Box(210, 160, 3), 'reward_range': (-inf, inf), 'metadata': {'render.modes': ['human', 'rgb_array']}, '_max_episode_seconds': None, '_max_episode_steps': 100000, '_elapsed_steps': 0, '_episode_started_at': None}


[0, 592379725]

In [3]:
class DQNReplayer:
    def __init__(self, capacity):
        self.memory = pd.DataFrame(index=range(capacity),
                columns=['observation', 'action', 'reward',
                'next_observation', 'done'])
        self.i = 0
        self.count = 0
        self.capacity = capacity
    
    def store(self, *args):
        self.memory.loc[self.i] = args
        self.i = (self.i + 1) % self.capacity
        self.count = min(self.count + 1, self.capacity)
        
    def sample(self, size):
        indices = np.random.choice(self.count, size=size)
        return tuple(np.stack(self.memory.loc[indices, field]) for \
                field in self.memory.columns)


class DQNAgent:
    def __init__(self, env, input_shape, learning_rate=0.00025,
            load_path=None, gamma=0.99,
            replay_memory_size=1000000, batch_size=32,
            replay_start_size=0,
            epsilon=1., epsilon_decrease_rate=9e-7, min_epsilon=0.1,
            random_inital_steps=0,
            clip_reward=True, rescale_state=True,
            update_freq=1, target_network_update_freq=1):
        
        self.action_n = env.action_space.n
        self.gamma = gamma
        
        self.replay_memory_size = replay_memory_size
        self.replay_start_size = replay_start_size
        self.batch_size = batch_size
        
        self.img_shape = (input_shape[-1], input_shape[-2])
        self.img_stack = input_shape[-3]
        
        self.epsilon = epsilon
        self.epsilon_decrease_rate = epsilon_decrease_rate
        self.min_epsilon = min_epsilon
        self.random_inital_steps = random_inital_steps
        
        self.clip_reward = clip_reward
        self.rescale_state = rescale_state
        
        self.update_freq = update_freq
        self.target_network_update_freq = target_network_update_freq
        
        self.replayer = DQNReplayer(replay_memory_size)
        
        self.step = 0
        self.fit_count = 0

        self.evaluate_net = self.build_network(
                input_shape=input_shape, output_size=self.action_n,
                conv_activation=tf.nn.relu,
                fc_hidden_sizes=[512,], fc_activation=tf.nn.relu,
                learning_rate=learning_rate, load_path=load_path)
        self.evaluate_net.summary() # print summary
        
        self.target_net = self.build_network(
                input_shape=input_shape, output_size=self.action_n,
                conv_activation=tf.nn.relu,
                fc_hidden_sizes=[512,], fc_activation=tf.nn.relu,
                )
        
        self.update_target_network()
        

    def build_network(self, input_shape, output_size, conv_activation,
            fc_hidden_sizes, fc_activation, output_activation=None,
            learning_rate=0.001, load_path=None):
        # network input: (samples, channels, rows, cols)
        model = keras.models.Sequential()
        # change from (channel, width, height) -> (width, height, channel)
        model.add(keras.layers.Permute((2, 3, 1), input_shape=input_shape))
        
        model.add(keras.layers.Conv2D(32, 8, strides=(4, 4),
                activation=conv_activation))
        model.add(keras.layers.Conv2D(64, 4, strides=(2, 2),
                activation=conv_activation))
        model.add(keras.layers.Conv2D(64, 3, strides=(1, 1),
                activation=conv_activation))
        
        model.add(keras.layers.Flatten())
        
        for hidden_size in fc_hidden_sizes:
            model.add(keras.layers.Dense(hidden_size,
                    activation=fc_activation))
        model.add(keras.layers.Dense(output_size,
                activation=output_activation))

        if load_path is not None:
            logging.info('Loading weights from file {}.'.format(load_path))
            model.load_weights(load_path)

        try: # tf2
            optimizer = keras.optimizers.RMSprop(learning_rate, 0.95,
                    momentum=0.95, epsilon=0.01)
        except: # tf1
            optimizer = tf.train.RMSPropOptimizer(learning_rate, 0.95,
                    momentum=0.95, epsilon=0.01)
        model.compile(loss=keras.losses.mse, optimizer=optimizer)
        return model
        
    def get_next_state(self, state=None, observation=None):
        img = Image.fromarray(observation, 'RGB') 
        img = img.resize(self.img_shape).convert('L') # To gray-scale and Resize
        img = np.asarray(img.getdata(), dtype=np.uint8
                ).reshape(img.size[1], img.size[0]) # Convert image to array
        
        if state is None:
            next_state = np.array([img,] * self.img_stack)
        else:
            next_state = np.append(state[1:], [img,], axis=0)
        return next_state
    
    def decide(self, state, test=False, step=None):
        if step is not None and step < self.random_inital_steps:
            epsilon = 1.
        elif test:
            epsilon = 0.05
        else:
            epsilon = self.epsilon
        if np.random.rand() < epsilon:
            action = np.random.choice(self.action_n)
        else:
            if self.rescale_state:
                state = state / 128. - 1.
            q_values = self.evaluate_net.predict(state[np.newaxis])[0]
            action = np.argmax(q_values)
        return action

    def learn(self, state, action, reward, next_state, done):
        self.replayer.store(state, action, reward, next_state, done)

        self.step += 1
        
        if self.step % self.update_freq == 0 and \
                self.replayer.count >= self.replay_start_size:

            states, actions, rewards, next_states, dones = \
                    self.replayer.sample(self.batch_size)

            if self.rescale_state:
                states = states / 128. - 1.
                next_states = next_states / 128. - 1.
            if self.clip_reward:
                rewards = np.clip(rewards, -1., 1.)
            
            next_qs = self.target_net.predict(next_states)
            next_max_qs = next_qs.max(axis=-1)
            targets = self.evaluate_net.predict(states)
            targets[range(self.batch_size), actions] = rewards + \
                    self.gamma * next_max_qs * (1. - dones)

            h = self.evaluate_net.fit(states, targets, verbose=0)
            self.fit_count += 1
            
            if self.fit_count % 100 == 0:
                logging.info(('fit-count : {}, epsilon = {}, ' +
                        'memory = {}, loss = {}').format(self.fit_count,
                        self.epsilon, self.replayer.count,
                        h.history['loss'][0]))
            
            if self.fit_count % self.target_network_update_freq == 0:
                self.update_target_network()
        
        # update_epsilon : Linear epsilon annealing
        # Decreases the probability of picking a random action,
        # to improve exploitation.
        if self.step >= self.replay_start_size:
            self.epsilon = max(self.epsilon - self.epsilon_decrease_rate,
                               self.min_epsilon)

    def update_target_network(self):
        logging.info('target network updated.')
        self.target_net.set_weights(self.evaluate_net.get_weights())

    def save_network(self, path):
        dirname = os.path.dirname(save_path)
        if not os.path.exists(dirname):
            os.makedirs(dirname)
            logging.info('create directory = {}'.format(dirname))
        self.evaluate_net.save_weights(path)
        logging.info('model saves to {}'.format(path))

In [4]:
def test(env, agent, episodes=100, render=False, verbose=True):
    steps, episode_rewards = [], []
    for episode in range(episodes):
        episode_reward = 0
        observation = env.reset()
        state = agent.get_next_state(None, observation)
        for step in itertools.count():
            if render:
                env.render()
            action = agent.decide(state, test=True, step=step)
            observation, reward, done, info = env.step(action)
            state = agent.get_next_state(state, observation)
            episode_reward += reward
            if done:
                break
        step += 1
        steps.append(step)
        episode_rewards.append(episode_reward)
        logging.info('[test] episode {}: step {}, reward {}, total_step {}'
                .format(episode, step, episode_reward, np.sum(steps)))
            
    if verbose:
        logging.info(('[test summary] step: mean = {}, min = {}, max = {}.'
                + ' episode reward: mean = {}, min = {}, max = {}').format(
                np.mean(steps), np.min(steps), np.max(steps),
                np.mean(episode_rewards), np.min(episode_rewards),
                np.max(episode_rewards)))
    return episode_rewards, steps

In [ ]:
render = False
load_path = None
save_path = './output/' + env.unwrapped.spec.id + '-' + \
        time.strftime('%Y%m%d-%H%M%S') + '/model.h5'

"""
Large-scale parameters: designated by Nature DQN
"""
input_shape = (4, 110, 84) # input_shape of network
batch_size = 32
replay_memory_size = 1000000
target_network_update_freq = 10000
gamma = 0.99
update_freq = 4 # frequency (number of steps) with which to train the DQN
learning_rate = 0.00025 # learning rate for optimizer
epsilon = 1. # initial exploration rate for the agent
min_epsilon = 0.1 # final exploration rate for the agent
epsilon_decrease = 9e-7 # rate at which to linearly decrease epsilon
replay_start_size = 50000 # minimum transitions before starting training
random_inital_steps = 30 # number of random actions to be performed by the agent at the beginning of each episode
frames = 50000000 # maximum number of frames during the whole algorithm
test_freq = 50000 # number of frames to test the agent's performance
test_episodes = 100

"""
Median-scale parameters : finish in days
"""
batch_size = 32
replay_memory_size = 100000
target_network_update_freq = 5000
replay_start_size = 20000
random_inital_steps = 30
frames = 1000000
test_freq = 40000
test_episodes = 100

"""
Small-scale parameters : finish in minutes
"""
# batch_size = 6
# replay_memory_size = 5000
# target_network_update_freq = 80
# replay_start_size = 500
# random_inital_steps = 30
# frames = 7500
# test_freq = 2500
# test_episodes = 10


agent = DQNAgent(env, input_shape=input_shape, batch_size=batch_size,
        replay_memory_size=replay_memory_size,        
        learning_rate=learning_rate, gamma=gamma,
        epsilon=epsilon, epsilon_decrease_rate=epsilon_decrease,
        min_epsilon=min_epsilon,
        random_inital_steps=random_inital_steps,
        load_path=load_path,
        update_freq=update_freq,
        target_network_update_freq=target_network_update_freq)

logging.info("training starts")

frame = 0
max_mean_episode_reward = float("-inf")
for episode in itertools.count():
    observation = env.reset()
    episode_reward = 0
    state = agent.get_next_state(None, observation)
    for step in itertools.count():
        if render:
            env.render()
        frame += 1
        action = agent.decide(state, step=step)
        observation, reward, done, _ = env.step(action)
        next_state = agent.get_next_state(state, observation)
        episode_reward += reward
        agent.learn(state, action, reward, next_state, done)
        
        if frame % test_freq == 0 or \
                (done and (frame + 1) % test_freq == 0):
            test_episode_rewards, test_steps = test(env=env,
                    agent=agent, episodes=test_episodes, render=render)
            if max_mean_episode_reward < np.mean(test_episode_rewards):
                max_mean_episode_reward = np.mean(test_episode_rewards)
                agent.save_network(save_path)
                path = save_path[:-2] + str(agent.fit_count) + '.h5'
                agent.save_network(path)
        
        if done:
            step += 1
            frame += 1
            break
        state = next_state
    
    logging.info("episode {}, step {}, reward {}, frame {}".format(
            episode, step, episode_reward, frame))
    
    if frame > frames:
        break

logging.info("training ends")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
permute (Permute)            (None, 110, 84, 4)        0         
_________________________________________________________________
conv2d (Conv2D)              (None, 26, 20, 32)        8224      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 12, 9, 64)         32832     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 10, 7, 64)         36928     
_________________________________________________________________
flatten (Flatten)            (None, 4480)              0         
_________________________________________________________________
dense (Dense)                (None, 512)               2294272   
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 3078      
Total para

2019-02-13 09:15:11,148 [INFO] fit-count : 4100, epsilon = 0.9852409000004859, memory = 16400, loss = 0.00013710367784369737
2019-02-13 09:15:36,347 [INFO] episode 25, step 680, reward 120.0, frame 16708
2019-02-13 09:15:47,273 [INFO] fit-count : 4200, epsilon = 0.9848809000004978, memory = 16800, loss = 0.0001744408073136583
2019-02-13 09:16:23,246 [INFO] fit-count : 4300, epsilon = 0.9845209000005096, memory = 17200, loss = 0.0052605741657316685
2019-02-13 09:16:44,947 [INFO] episode 26, step 759, reward 165.0, frame 17468
2019-02-13 09:16:59,957 [INFO] fit-count : 4400, epsilon = 0.9841609000005215, memory = 17600, loss = 0.0001078589993994683
2019-02-13 09:17:36,624 [INFO] fit-count : 4500, epsilon = 0.9838009000005333, memory = 18000, loss = 0.005181307904422283
2019-02-13 09:17:48,668 [INFO] episode 27, step 687, reward 135.0, frame 18156
2019-02-13 09:18:13,851 [INFO] fit-count : 4600, epsilon = 0.9834409000005452, memory = 18400, loss = 7.745139009784907e-05
2019-02-13 09:18:36

2019-02-13 09:47:49,404 [INFO] fit-count : 9100, epsilon = 0.9672409000010785, memory = 36400, loss = 0.005364275071769953
2019-02-13 09:48:32,543 [INFO] fit-count : 9200, epsilon = 0.9668809000010904, memory = 36800, loss = 0.00013514110469259322
2019-02-13 09:49:05,191 [INFO] episode 51, step 1442, reward 430.0, frame 37155
2019-02-13 09:49:15,577 [INFO] fit-count : 9300, epsilon = 0.9665209000011022, memory = 37200, loss = 0.004842420108616352
2019-02-13 09:49:57,998 [INFO] fit-count : 9400, epsilon = 0.9661609000011141, memory = 37600, loss = 0.005250522401183844
2019-02-13 09:50:13,653 [INFO] episode 52, step 642, reward 105.0, frame 37798
2019-02-13 09:50:41,135 [INFO] fit-count : 9500, epsilon = 0.965800900001126, memory = 38000, loss = 0.00022713519865646958
2019-02-13 09:51:23,188 [INFO] episode 53, step 644, reward 135.0, frame 38443
2019-02-13 09:51:24,546 [INFO] fit-count : 9600, epsilon = 0.9654409000011378, memory = 38400, loss = 0.0001077071501640603
2019-02-13 09:52:07,

2019-02-13 10:27:52,901 [INFO] [test] episode 75: step 902, reward 155.0, total_step 63245
2019-02-13 10:28:30,540 [INFO] [test] episode 76: step 1400, reward 455.0, total_step 64645
2019-02-13 10:28:55,644 [INFO] [test] episode 77: step 951, reward 110.0, total_step 65596
2019-02-13 10:29:21,069 [INFO] [test] episode 78: step 951, reward 390.0, total_step 66547
2019-02-13 10:29:36,580 [INFO] [test] episode 79: step 581, reward 50.0, total_step 67128
2019-02-13 10:30:00,254 [INFO] [test] episode 80: step 885, reward 230.0, total_step 68013
2019-02-13 10:30:19,931 [INFO] [test] episode 81: step 733, reward 80.0, total_step 68746
2019-02-13 10:30:47,070 [INFO] [test] episode 82: step 1043, reward 315.0, total_step 69789
2019-02-13 10:31:05,049 [INFO] [test] episode 83: step 665, reward 160.0, total_step 70454
2019-02-13 10:31:21,415 [INFO] [test] episode 84: step 617, reward 125.0, total_step 71071
2019-02-13 10:31:38,500 [INFO] [test] episode 85: step 641, reward 110.0, total_step 71712

2019-02-13 10:58:13,887 [INFO] episode 74, step 610, reward 155.0, frame 52763
2019-02-13 10:58:25,490 [INFO] fit-count : 13200, epsilon = 0.9524809000015645, memory = 52800, loss = 0.0001735777477733791
2019-02-13 10:59:04,429 [INFO] episode 75, step 481, reward 80.0, frame 53245
2019-02-13 10:59:07,900 [INFO] fit-count : 13300, epsilon = 0.9521209000015763, memory = 53200, loss = 0.00012492078531067818
2019-02-13 10:59:50,077 [INFO] fit-count : 13400, epsilon = 0.9517609000015882, memory = 53600, loss = 0.0002642504405230284
2019-02-13 11:00:17,299 [INFO] episode 76, step 681, reward 155.0, frame 53927
2019-02-13 11:00:33,497 [INFO] fit-count : 13500, epsilon = 0.9514009000016, memory = 54000, loss = 0.009859294630587101
2019-02-13 11:01:16,376 [INFO] fit-count : 13600, epsilon = 0.9510409000016119, memory = 54400, loss = 0.00016405939823016524
2019-02-13 11:01:29,873 [INFO] episode 77, step 675, reward 180.0, frame 54603
2019-02-13 11:01:58,964 [INFO] fit-count : 13700, epsilon = 0.

...

2019-02-17 15:41:21,097 [INFO] fit-count : 216600, epsilon = 0.22024090000933477, memory = 100000, loss = 0.0046622660011053085
2019-02-17 15:42:10,909 [INFO] fit-count : 216700, epsilon = 0.21988090000933552, memory = 100000, loss = 0.009748965501785278
2019-02-17 15:43:01,405 [INFO] episode 1208, step 1019, reward 210.0, frame 868400
2019-02-17 15:43:02,847 [INFO] fit-count : 216800, epsilon = 0.21952090000933627, memory = 100000, loss = 0.005359025206416845
2019-02-17 15:44:00,774 [INFO] fit-count : 216900, epsilon = 0.21916090000933702, memory = 100000, loss = 0.00013525273243431002
2019-02-17 15:44:01,333 [INFO] episode 1209, step 413, reward 50.0, frame 868814
2019-02-17 15:44:52,562 [INFO] fit-count : 217000, epsilon = 0.21880090000933777, memory = 100000, loss = 0.005580397322773933
2019-02-17 15:45:06,144 [INFO] episode 1210, step 497, reward 80.0, frame 869312
2019-02-17 15:45:45,757 [INFO] fit-count : 217100, epsilon = 0.21844090000933852, memory = 100000, loss = 0.000410085

2019-02-17 16:27:35,929 [INFO] [test] episode 33: step 748, reward 190.0, total_step 26435
2019-02-17 16:28:24,607 [INFO] [test] episode 34: step 1228, reward 515.0, total_step 27663
2019-02-17 16:28:53,373 [INFO] [test] episode 35: step 726, reward 225.0, total_step 28389
2019-02-17 16:29:23,538 [INFO] [test] episode 36: step 770, reward 320.0, total_step 29159
2019-02-17 16:29:41,807 [INFO] [test] episode 37: step 487, reward 30.0, total_step 29646
2019-02-17 16:30:05,241 [INFO] [test] episode 38: step 650, reward 175.0, total_step 30296
2019-02-17 16:30:21,564 [INFO] [test] episode 39: step 489, reward 45.0, total_step 30785
2019-02-17 16:30:42,029 [INFO] [test] episode 40: step 618, reward 105.0, total_step 31403
2019-02-17 16:31:02,383 [INFO] [test] episode 41: step 625, reward 130.0, total_step 32028
2019-02-17 16:31:26,711 [INFO] [test] episode 42: step 717, reward 185.0, total_step 32745
2019-02-17 16:31:54,884 [INFO] [test] episode 43: step 847, reward 105.0, total_step 33592


2019-02-17 17:11:36,172 [INFO] episode 1230, step 849, reward 130.0, frame 884641
2019-02-17 17:12:04,414 [INFO] fit-count : 220900, epsilon = 0.20476090000936703, memory = 100000, loss = 0.00015100670862011611
2019-02-17 17:13:05,046 [INFO] fit-count : 221000, epsilon = 0.20440090000936778, memory = 100000, loss = 0.006407805252820253
2019-02-17 17:13:15,287 [INFO] episode 1231, step 667, reward 115.0, frame 885309
2019-02-17 17:13:58,536 [INFO] fit-count : 221100, epsilon = 0.20404090000936853, memory = 100000, loss = 0.0046659205108881
2019-02-17 17:14:51,228 [INFO] episode 1232, step 676, reward 250.0, frame 885986
2019-02-17 17:14:57,082 [INFO] fit-count : 221200, epsilon = 0.20368090000936928, memory = 100000, loss = 0.0003241105587221682
2019-02-17 17:15:55,225 [INFO] fit-count : 221300, epsilon = 0.20332090000937003, memory = 100000, loss = 0.005595575086772442
2019-02-17 17:16:45,154 [INFO] episode 1233, step 781, reward 390.0, frame 886768
2019-02-17 17:16:54,490 [INFO] fit-c

2019-02-17 17:54:56,345 [INFO] episode 1257, step 516, reward 70.0, frame 903602
2019-02-17 17:55:03,105 [INFO] fit-count : 225600, epsilon = 0.1878409000094023, memory = 100000, loss = 0.00027121586026623845
2019-02-17 17:55:56,231 [INFO] fit-count : 225700, epsilon = 0.18748090000940304, memory = 100000, loss = 0.005484242923557758
2019-02-17 17:55:58,987 [INFO] episode 1258, step 479, reward 75.0, frame 904082
2019-02-17 17:56:49,140 [INFO] fit-count : 225800, epsilon = 0.1871209000094038, memory = 100000, loss = 0.004717764910310507
2019-02-17 17:57:27,265 [INFO] episode 1259, step 629, reward 215.0, frame 904712
2019-02-17 17:57:46,655 [INFO] fit-count : 225900, epsilon = 0.18676090000940454, memory = 100000, loss = 0.00022927657118998468
2019-02-17 17:58:41,492 [INFO] fit-count : 226000, epsilon = 0.1864009000094053, memory = 100000, loss = 0.000463955570012331
2019-02-17 17:59:34,930 [INFO] fit-count : 226100, epsilon = 0.18604090000940604, memory = 100000, loss = 0.000396694580

2019-02-17 18:38:25,975 [INFO] [test] episode 14: step 975, reward 355.0, total_step 13531
2019-02-17 18:38:42,353 [INFO] [test] episode 15: step 497, reward 80.0, total_step 14028
2019-02-17 18:39:19,720 [INFO] [test] episode 16: step 1109, reward 310.0, total_step 15137
2019-02-17 18:39:48,511 [INFO] [test] episode 17: step 861, reward 215.0, total_step 15998
2019-02-17 18:40:01,455 [INFO] [test] episode 18: step 399, reward 80.0, total_step 16397
2019-02-17 18:40:24,185 [INFO] [test] episode 19: step 647, reward 155.0, total_step 17044


In [ ]:
test_agent = DQNAgent(env, input_shape=input_shape, load_path=save_path)
test_episode_rewards, _ = test(env, test_agent, episodes=test_episodes)
print('mean(test_episode_rewards) = {}'.format(
        np.mean(test_episode_rewards)))